In [1]:
from typing import Tuple

import itertools

import pandas

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'canada'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
def train_model(df: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse', eval_metric_optimize: str = 'min') -> Prophet:
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, horizon = '60 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1, monthly=True)
        eval_metric_values.append(df_p[eval_metric].values[0])

    # Find the best parameters
    tuning_results = pandas.DataFrame(all_params)
    tuning_results[eval_metric] = eval_metric_values

    tuning_results.sort_values(by = eval_metric, axis = 'index', ascending = True, inplace = True)
    best_params = {}
    if eval_metric_optimize == 'min':
        best_params = tuning_results.iloc[0].to_dict()
    elif eval_metric_optimize == 'max':
        best_params = tuning_results.iloc[-1].to_dict()
    else:
        raise ValueError("Invalid Parameter Value: param 'eval_metric_optimize' may only have values 'min' or 'max'.")


    optimal_model = Prophet(**params).fit(df)
    return optimal_model

In [8]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

model = train_model(df_train, param_grid)

23:42:03 - cmdstanpy - INFO - Chain [1] start processing
23:42:03 - cmdstanpy - INFO - Chain [1] done processing
23:42:03 - cmdstanpy - ERROR - Chain [1] error: error during processing Communication error on send
Optimization terminated abnormally. Falling back to Newton.
23:42:03 - cmdstanpy - INFO - Chain [1] start processing
23:42:05 - cmdstanpy - INFO - Chain [1] done processing
23:42:07 - cmdstanpy - INFO - Chain [1] start processing
23:42:07 - cmdstanpy - INFO - Chain [1] start processing
23:42:07 - cmdstanpy - INFO - Chain [1] start processing
23:42:07 - cmdstanpy - INFO - Chain [1] start processing
23:42:07 - cmdstanpy - INFO - Chain [1] start processing
23:42:07 - cmdstanpy - INFO - Chain [1] start processing
23:42:07 - cmdstanpy - INFO - Chain [1] start processing
23:42:07 - cmdstanpy - INFO - Chain [1] start processing
23:42:07 - cmdstanpy - INFO - Chain [1] done processing
23:42:07 - cmdstanpy - INFO - Chain [1] done processing
23:42:07 - cmdstanpy - INFO - Chain [1] done p

In [9]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [10]:
df_predicted, rmse = test_model(df_test, model)

In [11]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-04-01,7.046081,6.129166,8.099950,7.046081,7.046081,0.092971,0.092971,0.092971,0.092971,0.092971,0.092971,0.0,0.0,0.0,7.139053
1,2022-05-01,7.044854,6.120237,8.040249,7.044854,7.044854,0.078478,0.078478,0.078478,0.078478,0.078478,0.078478,0.0,0.0,0.0,7.123332
2,2022-06-01,7.043585,6.064942,8.118833,7.043585,7.043585,0.047485,0.047485,0.047485,0.047485,0.047485,0.047485,0.0,0.0,0.0,7.091070
3,2022-07-01,7.042357,6.057974,7.986723,7.042357,7.042357,0.030646,0.030646,0.030646,0.030646,0.030646,0.030646,0.0,0.0,0.0,7.073003
4,2022-08-01,7.041088,6.062225,8.038290,7.041088,7.041088,-0.003760,-0.003760,-0.003760,-0.003760,-0.003760,-0.003760,0.0,0.0,0.0,7.037328
5,2022-09-01,7.039819,6.014291,8.077083,7.039819,7.039819,-0.012833,-0.012833,-0.012833,-0.012833,-0.012833,-0.012833,0.0,0.0,0.0,7.026986
6,2022-10-01,7.038591,5.993260,7.964202,7.038591,7.051205,-0.023113,-0.023113,-0.023113,-0.023113,-0.023113,-0.023113,0.0,0.0,0.0,7.015478
7,2022-11-01,7.037322,6.035768,8.072844,7.037322,7.068002,-0.033395,-0.033395,-0.033395,-0.033395,-0.033395,-0.033395,0.0,0.0,0.0,7.003927
8,2022-12-01,7.036095,5.993596,8.020890,7.033494,7.090098,-0.038082,-0.038082,-0.038082,-0.038082,-0.038082,-0.038082,0.0,0.0,0.0,6.998013
9,2023-01-01,7.034826,5.927197,8.080561,7.024183,7.109403,-0.009665,-0.009665,-0.009665,-0.009665,-0.009665,-0.009665,0.0,0.0,0.0,7.025161


In [12]:
rmse

1.9688710637326872

In [13]:
with open(f'{country}_prophet_optimized_model.json', 'w') as f:
    f.write(model_to_json(model))

In [14]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [15]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [16]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,7.032411,6.088547,8.081462,7.032411,7.032411,0.087082,0.087082,0.087082,0.087082,0.087082,0.087082,0.0,0.0,0.0,7.119493
1,2023-04-01,7.031142,6.126887,8.070385,7.031142,7.031142,0.069863,0.069863,0.069863,0.069863,0.069863,0.069863,0.0,0.0,0.0,7.101005
2,2023-05-01,7.029914,6.047840,8.051687,7.029914,7.029914,0.043067,0.043067,0.043067,0.043067,0.043067,0.043067,0.0,0.0,0.0,7.072981
